<a href="https://colab.research.google.com/github/johntango/shakespeare001/blob/main/ShakespeareColab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [41]:
%%bigquery --project demoproject002-351320 --verbose df
SELECT * FROM `demoproject002-351320.Shakespeare.shakespeareT8`

Executing query with job ID: 2b6d1027-ab62-497e-b93b-29b451791e91
Query executing: 13.45s
Query complete after 13.82s


In [43]:
df.columns

Index(['text'], dtype='object')

In [28]:
#df.isnull().sum()

#df1 = df[df['string_field_3'].isnull() == False]
print(df.count())
df.fillna('', inplace=True)
df['text']= df[['string_field_0','string_field_1','string_field_2','string_field_3']].agg(' '.join, axis=1)
df1 = df.drop(['string_field_0','string_field_1','string_field_2','string_field_3'],axis=1)
df1.count()
df1.columns

string_field_0    116865
string_field_1    116865
string_field_2    116865
string_field_3    116865
text              116865
dtype: int64


Index(['text'], dtype='object')

In [29]:
# drop columns and clean up rows with none
import pandas as pd

df3 = df1.dropna()
df3.head

<bound method NDFrame.head of                                                      text
0                                          THE SONNETS   
1                                                        
2                               by William Shakespeare   
3                                                        
4                                                        
...                                                   ...
116860  SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR ...
116861                                                   
116862                                                   
116863                                                   
116864  End of this Etext of The Complete Works of Wil...

[116865 rows x 1 columns]>

In [30]:
# covert caps to lower case and clean the punctuation 
ds=df3['text'].str.lower().str.replace(r'[^\w\s]+', '')
df4 = pd.DataFrame({'text':ds})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [31]:
# now split the strings into lists 
df5 = df4.assign(text=df4.text.str.split(" "))
df5.head

<bound method NDFrame.head of                                                      text
0                                    [the, sonnets, , , ]
1                                                [, , , ]
2                        [by, william, shakespeare, , , ]
3                                                [, , , ]
4                                                [, , , ]
...                                                   ...
116860  [service, that, charges, for, download, time, ...
116861                                           [, , , ]
116862                                           [, , , ]
116863                                           [, , , ]
116864  [end, of, this, etext, of, the, complete, work...

[116865 rows x 1 columns]>

In [37]:
# now explode the lists into separate rows
df6 = df5.explode('text').reset_index(drop=True)
df6.dropna(inplace=True)
df6.head

<bound method NDFrame.head of                 text
0                the
1            sonnets
2                   
3                   
4                   
...              ...
1182848      william
1182849  shakespeare
1182850             
1182851             
1182852             

[1182853 rows x 1 columns]>

In [38]:
# now count the words
final_df = df6.value_counts().reset_index()
final_df.columns = ['word','count']
final_df.head

<bound method NDFrame.head of              word   count
0                  337964
1             the   25944
2             and   25065
3               i   19263
4              to   17939
...           ...     ...
27552  farborough       1
27553    priscian       1
27554         fap       1
27555     prisond       1
27556    zwaggerd       1

[27557 rows x 2 columns]>